# Overview of quantum simulators


In this notebook, we give a basic description of the 5 ideal quantum circuit simulators available on the Quantum Learning Machine.

The QLM also provides **noisy** simulation capabilities. For more details see [Setting up a noisy quantum computation](../noisy_simulation/overview.ipynb).


## What "simulation" means:

- By "simulation of quantum circuits", we mean one of the following tasks: <br><br>
    - **state analysis**: the simulator returns all non-zero amplitudes (default).
    
    - **strict emulation of quantum computing**: the simulator returns measurement results, sampled from the output probability distribution of the quantum circuit, just like an actual (ideal) quantum computer would. They can be regrouped by value or not, depending on the value assigned to the "aggregate_data" argument. See [this notebook](simulation_overview.ipynb) for more details.<br><br>
    - **computation of the average value of an observable** You can directly ask a simulator for an observable average. In terms of performance, this more efficient than getting the amplitudes as a numpy array ([which is possible, see this notebook](simulation_overview.ipynb)) and computing it yourself.


## Quick description of the simulators:

Each of the 5 ideal simulators is based on particular data structures, which make it behave well with specific families of quantum circuits.

- The different ideal simulators are the following: <br><br>
    - **qat.linalg**: a simulator based on straightforward linear application of unitary matrices onto the amplitude vector. Both its memory usage and execution time are exponential in the number of qubits. <br><br>
    *Restrictions*: None. Any circuit can be run on this simulator. <br><br>
    
    - **qat.feynman**: a simulator based on the Feynman path integral formulation of quantum computing (see [Rudiak-Gould06](https://arxiv.org/abs/quant-ph/0607151) for instance). Its execution time is exponential in the number of *dense* gates. A gate is dense if it is neither logical (like CNOT or Toffoli - containing one 1 per line and column) nor diagonal. As qat.linalg, its memory usage is exponential in the number of qubits, as it still computes the entire amplitude vector.<br><br>
    *Restrictions:* gates are limited to arity < 4.
    <br><br>
    
    - **qat.mps**: a simulator where the state vector is stored as a Matrix Product State. This data structure compresses information in slightly entangled cases. In these cases, both memory usage and execution time are greatly reduced. Some references on MPS simulation of quantum computing: [Vidal03](https://arxiv.org/abs/quant-ph/0301063) or [Banuls05](https://arxiv.org/abs/quant-ph/0503174) <br><br>
    *Restrictions:* gates are limited to arity < 4 and need to act on neighboring qubits (*i-e* with consecutive qubit indices).
    <br><br>
    
    - **qat.stabs**: a simulator based on the stabilizer formalism ([Gottesman97](https://arxiv.org/abs/quant-ph/9705052), [Aaronson04](https://arxiv.org/abs/quant-ph/0406196)). It is, by construction, restricted to circuits containing Clifford gates (H, CNOT, S and the gates they generate) only. However, on these circuits, it can push the simulation to thousands of qubits.<br><br>
    *Restrictions:* only Clifford gates can be used (CNOT, H, S and everything they generate)<br><br>
    - **qat.bdd**: Similarly to MPS, its memory footprint will depend on the structure of the state being represented. It can be greatly reduced in cases where the state vector can "factorized". see [this article](https://arxiv.org/abs/1707.00865) for more details.

$\rightarrow$ This notebook will demonstrate the execution of some circuits on these different simulators, with the purpose of highlighting their respective weaknesses and strengths.<br>

## Testbeds

- The different circuits we will use to test the ideal simulators are:
    - The Quantum Fourier Transform, with 20 qubits.
    - An instance of a quantum arithmetics circuit (typical subpart of Shor's algorithm).
    - A cat state generation circuit with many qubits.<br><br>
    
- For noisy simulations, see [this notebook](../noisy_simulation/overview.ipynb).



## Quantum Fourier Transform from computational basis state.

Let us start with a QFT, which we import from our programming libraries:

In [1]:
from qat.lang.AQASM import *
from qat.lang.AQASM.qftarith import QFT

In [2]:
nqbits = 5
prog = Program()
reg = prog.qalloc(nqbits)
prog.apply(QFT(nqbits), reg)
qft_c = prog.to_circ()

**Let us now run this QFT circuit on the different simulators:**

In [3]:
#Running on linalg.
from qat.qpus import LinAlg

job = qft_c.to_job(nbshots=5, aggregate_data=False)

qpu = LinAlg()

results = qpu.submit(job)

for sample in results:
    print(sample)


Sample(_state=10, probability=None, _amplitude=None, intermediate_measurements=None, err=None, qregs=[QRegister(scope=<qat.lang.AQASM.program.Program object at 0x14b1238d0438>, length=5, start=0, msb=None, qbits=[<qat.lang.AQASM.bits.Qbit object at 0x14b123880358>, <qat.lang.AQASM.bits.Qbit object at 0x14b1238803c8>, <qat.lang.AQASM.bits.Qbit object at 0x14b123880438>, <qat.lang.AQASM.bits.Qbit object at 0x14b1238804a8>, <qat.lang.AQASM.bits.Qbit object at 0x14b123880518>])])
Sample(_state=24, probability=None, _amplitude=None, intermediate_measurements=None, err=None, qregs=[QRegister(scope=<qat.lang.AQASM.program.Program object at 0x14b1238d0438>, length=5, start=0, msb=None, qbits=[<qat.lang.AQASM.bits.Qbit object at 0x14b123880358>, <qat.lang.AQASM.bits.Qbit object at 0x14b1238803c8>, <qat.lang.AQASM.bits.Qbit object at 0x14b123880438>, <qat.lang.AQASM.bits.Qbit object at 0x14b1238804a8>, <qat.lang.AQASM.bits.Qbit object at 0x14b123880518>])])
Sample(_state=27, probability=None, _a

In [4]:
#Running on feynman=path-integral based.
from qat.feynman import Feynman

qpu = Feynman()

results = qpu.submit(job)

for sample in results:
    print(sample)

Sample(_state=14, probability=None, _amplitude=None, intermediate_measurements=None, err=None, qregs=[QRegister(scope=<qat.lang.AQASM.program.Program object at 0x14b1238906a0>, length=5, start=0, msb=None, qbits=[<qat.lang.AQASM.bits.Qbit object at 0x14b123890b38>, <qat.lang.AQASM.bits.Qbit object at 0x14b123890ba8>, <qat.lang.AQASM.bits.Qbit object at 0x14b123890c18>, <qat.lang.AQASM.bits.Qbit object at 0x14b123890c88>, <qat.lang.AQASM.bits.Qbit object at 0x14b123890cf8>])])
Sample(_state=12, probability=None, _amplitude=None, intermediate_measurements=None, err=None, qregs=[QRegister(scope=<qat.lang.AQASM.program.Program object at 0x14b1238906a0>, length=5, start=0, msb=None, qbits=[<qat.lang.AQASM.bits.Qbit object at 0x14b123890b38>, <qat.lang.AQASM.bits.Qbit object at 0x14b123890ba8>, <qat.lang.AQASM.bits.Qbit object at 0x14b123890c18>, <qat.lang.AQASM.bits.Qbit object at 0x14b123890c88>, <qat.lang.AQASM.bits.Qbit object at 0x14b123890cf8>])])
Sample(_state=4, probability=None, _am

In [5]:
# Running on MPS simulator, on even more qubits.
from qat.mps import MPS 

nqbits = 50
prog = Program()
reg = prog.qalloc(nqbits)
prog.apply(QFT(nqbits), reg)
qft_c = prog.to_circ()

job = qft_c.to_job(nbshots=5) # Note that we are leaving aggregate_data to default True

# Since evaluating the true memory requirement of qat-mps is a
# very difficult problem, our current memory model will sometimes
# reject a legitimate computation.  Setting
# disable_resource_management to True allows to override the model,
# at the cost of losing guarantees about memory.
qpu = MPS(lnnize=True, disable_resource_management=True) # making circuit MPS-compatible. 

res = qpu.submit(job)

for sample in res:
    print(sample)

Sample(_amplitude=None, probability=0.2, _state=261103782344905, err=0.2, intermediate_measurements=None, qregs=[QRegister(scope=<qat.lang.AQASM.program.Program object at 0x14b11b71fb70>, length=50, start=0, msb=None, qbits=[<qat.lang.AQASM.bits.Qbit object at 0x14b11b71fdd8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b71fe48>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b71feb8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b71ff28>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b71ff98>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b71fb00>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b701c18>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b701e48>, <qat.lang.AQASM.bits.Qbit object at 0x14b174402cf8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b705470>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b705588>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b705630>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b705748>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7057f0>, <qat.lang.AQASM.bits.Qbit object 

In [6]:
# Running on stabs, even though we know it won't work.
from qat.stabs import Stabs

try:
    qpu = Stabs()
    qpu.submit(job)
except:
    print("Expected Exception since Stabs support only Clifford gates")

Expected Exception since Stabs support only Clifford gates


### Observations:
- As expected, the QFT does not run on the stabilizer simulator, because it contains non-Clifford gates <br><br>
- The MPS simulator needs an option called "lnnize": by construction, it only acceps circuits containing gates acting on neighboring qubits. The lnnize options tells the simulator to insert SWAP gates in the circuit, in order to make it compliant with this toplogical constraint <br><br>
- You could also make the circuit compliant beforehand, by calling the qat-nnizer module on it. More work for optimization has been put into it, and it should return better results for large nnization tasks. However, it will also be slower, and it might be faster overall to call the simpler built-in nnizer, and spend computation time on actual simulation. <br><br>
- The Feynman (path integral-based) and linear algebra simulators are flexible enough to accept the circuit as it is. However, because we start from $\left|0...0\right>$, and the QFT does not produce much entanglement, MPS is faster and uses less memory.

The advantage of MPS on the QFT is best shown by running a 50 qubit example, which is unconceivable on linalg or feynman.

## Quantum arithmetics: Toffoli-based adder

Being a "classical circuit", in the sense that it sends computational basis states to other computational basis states, it should highlight the potential of the path-integral simulator (qat.feynman).

It is also slightly entangling, so the MPS simulator should also behave nicely on this example.

Such arithmetics circuits are typically used in Shor's algorithm. Simulating them as we do here could for instance be useful for debugging purposes, before inserting them into more complicated routines.

In [7]:
from qat.lang.AQASM.classarith import add

nqbits = 20
prog = Program()
reg = prog.qalloc(nqbits)
for qb in reg[::2]:
    prog.apply(X, qb)     # slightly modifying the input to check the output.
prog.apply(add(10,10), reg)     # add(5) takes a register of size 5 and adds its state 
                            # to a register of size 5 + 1.
                            # it also uses a third register of 5 qbits.
                            # hence the 16 = 5 + 1 + 5 + 5 qbits
                            # By convention, the state of the second register 
                            # (qb 5 to 9) is added to the first (qb 0 to 5).
        
adder_c = prog.to_circ()
%jsqatdisplay adder_c

UsageError: Line magic function `%jsqatdisplay` not found.


In [8]:
job = adder_c.to_job(nbshots=7,aggregate_data=False)

qpu = MPS(lnnize=True)

res = qpu.submit(job)

for sample in res:
    print(sample)

Sample(_amplitude=None, probability=None, _state=349866, err=None, intermediate_measurements=[], qregs=[QRegister(scope=<qat.lang.AQASM.program.Program object at 0x14b11b7d0780>, length=20, start=0, msb=None, qbits=[<qat.lang.AQASM.bits.Qbit object at 0x14b11b7d09e8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c83c8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7d0ac8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c8438>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7d0ba8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c84a8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7d0c88>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c8518>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7d0d68>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c8588>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7d0e48>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c85f8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7d0f28>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c8668>, <qat.lang.AQASM.bits.Qbit object at 0x14b1

In [9]:
qpu = Feynman()

res = qpu.submit(job)

for sample in res:
    print(sample)

Sample(_state=349866, probability=None, _amplitude=None, intermediate_measurements=None, err=None, qregs=[QRegister(scope=<qat.lang.AQASM.program.Program object at 0x14b11b7eff98>, length=20, start=0, msb=None, qbits=[<qat.lang.AQASM.bits.Qbit object at 0x14b11b7c2390>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7effd0>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c2470>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7efc50>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c2550>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7efb38>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c2630>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7ef9e8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c2710>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7efa90>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c27f0>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7efcc0>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7c28d0>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7efc88>, <qat.lang.AQASM.bits.Qbit object at 0x14

In [10]:
from qat.bdd import Bdd

qpu = Bdd()

res = qpu.submit(job)

for sample in res:
    print(sample)

Sample(_state=349866, probability=None, _amplitude=None, intermediate_measurements=None, err=None, qregs=[QRegister(scope=<qat.lang.AQASM.program.Program object at 0x14b11b727ac8>, length=20, start=0, msb=None, qbits=[<qat.lang.AQASM.bits.Qbit object at 0x14b11b727e48>, <qat.lang.AQASM.bits.Qbit object at 0x14b1238767b8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b727f28>, <qat.lang.AQASM.bits.Qbit object at 0x14b123876828>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b7efd30>, <qat.lang.AQASM.bits.Qbit object at 0x14b123876898>, <qat.lang.AQASM.bits.Qbit object at 0x14b1238760b8>, <qat.lang.AQASM.bits.Qbit object at 0x14b123876908>, <qat.lang.AQASM.bits.Qbit object at 0x14b123876198>, <qat.lang.AQASM.bits.Qbit object at 0x14b123876978>, <qat.lang.AQASM.bits.Qbit object at 0x14b123876278>, <qat.lang.AQASM.bits.Qbit object at 0x14b1238769e8>, <qat.lang.AQASM.bits.Qbit object at 0x14b123876358>, <qat.lang.AQASM.bits.Qbit object at 0x14b123876a58>, <qat.lang.AQASM.bits.Qbit object at 0x14

### Observations:
- As expected, qat.feynman is the fastest method for this kind of circuit. This is because there is only "one path in the integral".  <br><br>
- The MPS simulator also behaves quite well, indicating that an adder has low entangling power. Results would be different with a more entangled input state. <br><br>


## An example of pure Clifford circuit: Noisy Steane-encoded qubit.

Of course, if only Clifford gates are used, the stabs simulator, based on the stabilizer formalism, is the most reasonable choice. <br>

Such circuits emerge when working with quantum error correction, for instance.<br>

Then, if noise is modeled as random Pauli gates insertion (which can be far from being realistic), you could still use this simulator.<br>

The following piece of code presents the simulation of a noisy encoding of a state into the Steane code. 

The logical state to prepare is just $|+_{L}\rangle$, and the starting state is $|0\rangle^{\otimes\cdot n_{\text{physical qubits}}}$. 

Noise is naively modeled by random Pauli gate insertions in the encoding circuit. 

In [11]:
import numpy as np

n_logical = 45

prog = Program()

reg = prog.qalloc(7*n_logical)

pairs = [(0,3),(0,5),(0,6),
         (1,3),(1,4),(1,6),
         (2,3),(2,4),(2,5)]

p = 0.1


In [12]:
# Actual noisy encoding circuit:
                
for qb in range(n_logical):
    # preparation in the logical |+> state:
    
    # 1. H on every physical qb.
    for phys_qb in range(7*qb,7*(qb+1)):
        prog.apply(H,reg[phys_qb])
    
    # random Pauli gate
        if np.random.random() < p:
            noise = np.random.choice([X,Y,Z])
            prog.apply(noise, reg[qb]) # only works because 1 unique register.
    
    
    
    # 2. CZ on some pairs.
    for ph_qb_1, ph_qb_2 in pairs:
        ctrl = 7*qb + ph_qb_1
        targ = 7*qb + ph_qb_2
        prog.apply(CSIGN, [reg[ctrl],reg[targ]])

    for phys_qb in range(7*qb,7*(qb+1)):
        if np.random.random() < p:
            noise = np.random.choice([X,Y,Z])
            prog.apply(noise, reg[qb]) # only works because 1 unique register.
        
    # 3. Local Cliffords: H on some qubits.
    for phys_qb in range(7*qb,7*qb+4):
        prog.apply(H, reg[phys_qb])
        if np.random.random() < p:
            noise = np.random.choice([X,Y,Z])
            prog.apply(noise, reg[qb]) # only works because 1 unique register.
        
noisy_qec = prog.to_circ()

print("number of qbits: ", noisy_qec.nbqbits)
print("number of gates: ", len(noisy_qec.ops))

number of qbits:  315
number of gates:  968


In [13]:
from qat.stabs import Stabs

job = noisy_qec.to_job(nbshots=1)

qpu = Stabs()

res = qpu.submit(job)

for sample in res:
    print(sample)

Sample(_state=4925384024683308513, probability=1.0, _amplitude=None, intermediate_measurements=None, err=nan, qregs=[QRegister(scope=<qat.lang.AQASM.program.Program object at 0x14b12385c4a8>, length=315, start=0, msb=None, qbits=[<qat.lang.AQASM.bits.Qbit object at 0x14b12385c710>, <qat.lang.AQASM.bits.Qbit object at 0x14b12385c7f0>, <qat.lang.AQASM.bits.Qbit object at 0x14b12385c898>, <qat.lang.AQASM.bits.Qbit object at 0x14b12385c9e8>, <qat.lang.AQASM.bits.Qbit object at 0x14b12385ca58>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b64c710>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b64c4a8>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b64cfd0>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b653128>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b653208>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b6532b0>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b653390>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b6534e0>, <qat.lang.AQASM.bits.Qbit object at 0x14b11b6535c0>, <qat.lang.AQASM.bits.Qbit ob

### Observations:
   - This circuit can only run on the stab simulator. <br>